In [1]:
#import libraries for task
import numpy as np
import matplotlib.pyplot as plt

import lab_funcs as lf
import lab_funcs_disp as lfd
import my_funcs as mf

# Вычисление рядов

## Khan-sum

### Задания

1. Объясните различие в погрешностях при различных порядках суммирования.
3. Почему алгорит Кэхэна имеет значительно лучшую точность, чем последовательное суммирование?
3. Получим ли мы те же значения погрешностей, если будем суммировать последовательность со слагаемыми разных знаков? Проверьте на следующей последовательности:
$$x_k=\sin k.$$
4. Что произойдет с погрешностью, если элементы выборки с разными знаками упорядочить по возрастанию? По возрастанию абсолютной величины? Проверьте экспериментально.


### Решение

1. Если складывать суммы не учитывая порядок, то мы получим суммы где складываются большие числа с маленькими и погрешность увеличивается за счет отбрасывания нижних порядков.
Если складывать поочередно, то погрешность при каждом суммирование будет меньше, т.к. мы будем складывать относительно равные числа (равного порядка) и погрешность не будет такой большой как в первом случае, т.к. нижние порядки учитваются.
Продемострируем это на практике.

In [2]:
#generate data
K = 7
perm_data = lf.samples(K)
sort_data = lf.sort_samples(K)
sum_perm = lf.direct_sum(perm_data)
sum_sort = lf.direct_sum(sort_data)
cur_sum = lf.exact_sum(K)

print("Relative permut data error: ", lf.relative_error(cur_sum, sum_perm))
print("Relative sort data error: ", lf.relative_error(cur_sum, sum_sort))


Relative permut data error:  4.6550208134767723e-11
Relative sort data error:  1.9908519277181958e-11


2. Алгоритм Кэхэна не отбрасывает остаток, он его сохраняет на каждом шаге суммирования. И результат следующего суммирования будет брать во внимание остаток от предыдущего суммирования и создавать остаток для нового суммирования.
При суммирование по возрастанию мы уменьшили погрешность, но не учитывали остаток при суммировании.

Получим сумму для алгоритма Кэхэна

In [3]:
kahan_sum = lf.Kahan_sum(perm_data)
print("Relative Kahan sum error: ", lf.relative_error(cur_sum, kahan_sum))

Relative Kahan sum error:  0.0


3. При суммирование знакочередующегося ряда мы получаем погрешность близкую к машинной. Для алгоритма Кэхэна ничего принципиально не изменится если будем суммировать слагаемые с разными знаками.

In [8]:
N = 1000
correct_sin_sum = mf.get_correct_sin_sum(N)
sin_perm_data = mf.get_sin_perm_samples(N)
sum_sin_perm = mf.Kahan_sum(sin_perm_data)

print("Relative sum error to Kahan permut sin data: ", lf.relative_error(correct_sin_sum, sum_sin_perm))

Relative sum error to Kahan permut sin data:  9.54772862162757e-16


4. Если упорядочивать элементы выборки, то относительная погрешность будет уменьшатся. Лучшим вариантом упорядочивания является $\rightarrow$ упорядочивание не по абсолютной величине.

In [10]:

correct_sin_sum = mf.get_correct_sin_sum(N)
sin_sort_data = mf.get_sin_sort_samples(N)
sin_abs_sort_data = mf.get_sin_abs_sort_samples(N)

sum_sin_sort = mf.Kahan_sum(sin_sort_data)
sum_abs_sin_sort = mf.Kahan_sum(sin_abs_sort_data)

print("Relative sum error to Kahan permut sin data: ",
       lf.relative_error(correct_sin_sum, sum_sin_perm))
print("Relative sum error to Kahan sort sin data: ",
       lf.relative_error(correct_sin_sum, sum_sin_sort))
print("Relative sum error to Kahan abs sort sin data: ",
       lf.relative_error(correct_sin_sum, sum_abs_sin_sort))

Relative sum error to Kahan permut sin data:  9.54772862162757e-16
Relative sum error to Kahan sort sin data:  4.0918836949832464e-16
Relative sum error to Kahan abs sort sin data:  5.455844926644328e-16


## Вычисление дисперсии

### Задания
1. Обьясните, почему формулы оценки дисперсии имеют разные погрешности, хотя чтобы их применить, нужно выполнить одни и те же действия, но в разном порядке? Оцените погрешности обоих формул.
2. Предложите однопроходную формулу для оценки мат. ожидания и дисперсии, основанную на первой формуле для дисперсии. Воспользуйтесь компенсационным суммированием, чтобы увеличить точность. Попробуйте увеличить точность вычисления по сравнению со второй формулой хотя бы на два порядка.

### Решение
## 1. 
Для объяснения сначала оценим погрешности, после промоделируем это на примере. Из этого сложиться объяснение.



1) $$D[X]\approx\frac1N\sum_{n=1}^N\left(x_n-\frac1N\sum_{n=1}^Nx_n\right)^2.$$

| $f$ | $\delta x$ | $\Delta x$ |
|:-:|:-:|:-:|
| $x_n$ | $\epsilon$ | $\epsilon \cdot x$ |
| $\dfrac{\sum^{N}_{n} x_n}{N}$ | $\dfrac{N}{N} \cdot \epsilon = \epsilon$| $E(x)\cdot \epsilon$|
|$\bigg(x_n - \dfrac{1}{N}\sum_{n=1}^{N}x_n\bigg)^2$|$2\dfrac{2 \cdot E(x)}{D(x)^{0.5} / 2 } \cdot \epsilon$|$8 \epsilon \cdot E(x)\cdot D(x)^{0.5}$|
|$\dfrac{1}{N}\sum_{n=1}^{N}\bigg(x_n - \dfrac{1}{N}\sum_{n=1}^{N}x_n\bigg)^2$|$8\epsilon \dfrac{E(x)}{D(x)^{0.5}}$|$8 \epsilon \cdot E(x)\cdot D(x)^{0.5}$ |



2) $$D[X]\approx \frac1N\sum_{n=1}^N x_n^2-\left(\frac1N\sum_{n=1}^Nx_n\right)^2.$$
| $f$ |$\delta x$ | $\Delta x$ |
|:-:|:-:|:-:|
|$x_n$|$\epsilon$|$E(x)\cdot \epsilon$|
|$\bigg(\dfrac{1}{N}\sum_{n=1}^{N}x_n\bigg)^2$ |$2 \epsilon$ | $2 \epsilon \cdot E(x)^2$ |
|$\dfrac{1}{N}\sum_{n=1}^{N}x_n^2$ |$\dfrac{N}{N}\cdot 2\epsilon$ | $2\epsilon\cdot E(x)^2$ |
|$\dfrac{1}{N}\sum_{n=1}^{N}x_n^2 - \bigg(\dfrac{1}{N}\sum_{n=1}^{N}x_n\bigg)^2$| $2\epsilon + 2\epsilon$|$4 \epsilon \cdot E(x)^2$| 

Как мы видим, погрешность не зависит от N, покажем это на примере. Построим график значения дисперсии от кол-ва элементов $\rightarrow$ N

In [ ]:
N_max = 1000
N_array = mf.get_N_array(N_max)

x_arrayes = lfd.array_samples(N_max)
cur_disp = lfd.exact_variance()

disp_one_sqrt_f = np.full(N_max, -1.0)
disp_one_sqrt_s = np.full(N_max, -1.0)

for num_of_array in range(0, N_max):
    disp_one_sqrt_f[num_of_array] = lfd.direct_first_var(x_arrayes[num_of_array])
    disp_one_sqrt_s[num_of_array] = lfd.direct_second_var(x_arrayes[num_of_array])

plt.plot(N_array, disp_one_sqrt_f,".k")
plt.plot(N_array, disp_one_sqrt_f, ".b")
plt.plot(N_array, [cur_disp]*len(N_array),"--r") 
plt.xlabel("$Количество\; элементов\; в\; выборке\; N$")
plt.ylabel("$Значение\; дисперсии$")
plt.legend(["Первая формула", "Вторая формула", "Текущее значение дисперсии"])
plt.show() 

## 2. 
Получим однопроходную формулу:

$$N\cdot D_N - (N-1)\cdot D_{N-1} = \sum^N_k(x_k-\frac{1}{N} \sum^N_n x_n)^2 - \sum^{N-1}_k (x_k - \frac{1}{N-1} \sum^{N-1}_n x_n)^2 =$$
$$= (x_N-E_N)^2 + \sum^{N-1}_n ((x_n-E_N)^2 - (x_n-E_{N-1})^2) = (x_N-E_N)^2 + \sum^{N-1}_n (2x_n - E_N - E_{N-1})(E_{N-1} - E_N) $$

Тогда получим:

$$ D_N = \bigg( 1- \frac{1}{N} \bigg) D_{N-1} + \frac{1}{N} (x_N-E_N) (x_N-E_{N-1}) $$

Посчитаем и сравним для каждой формулы по дисперсии

In [1]:

x = lfd.samples(100000)
eps0 = np.finfo(np.double).eps
n = float(len(x))
mean = lfd.get_mean()
delta = lfd.get_delta()
variance = lfd.exact_variance()

theor_1 = 8*eps0*mean*delta**(0.5)
theor_2 = 4*eps0*mean**2

x_oneline_test=[1/2,4/7,3/5,11/19,5/19]
d1=lfd.oneline_first_var(x_oneline_test)
d2=lfd.online_second_var(x_oneline_test)
print('Сравнение однопроходных формул для первой и второй оценки дисперсии: разность получаемых значений',np.abs(d1-d2))

print("Размер выборки:", len(x))
print("Среднее значение:", lfd.exact_mean())
print("Оценка дисперсии:", lfd.exact_variance())
print("Ошибка среднего для встроенной функции:", lfd.relative_error(mean,np.mean(x)))
print("Ошибка дисперсии для встроенной функции:", lfd.relative_error(variance,np.var(x)))


print("Ошибка среднего для последовательного суммирования:", lfd.relative_error(mean,lfd.direct_mean(x)))

print("Ошибка второй оценки дисперсии для последовательного суммирования:",lfd.relative_error(variance,lfd.direct_second_var(x)))
print("Ошибка второй оценки дисперсии для однопроходного суммирования:",lfd.relative_error(variance,lfd.online_second_var(x)))


print("Ошибка первой оценки дисперсии для последовательного суммирования:",lfd.relative_error(variance,lfd.direct_first_var(x)))
print("Ошибка первой оценки дисперсии для однопроходного суммирования:",lfd.relative_error(variance,lfd.oneline_first_var(x)))


print("Теоретическая оценка ошибки дисперсии (по первой формуле):",theor_1)
print("Теоретическая оценка ошибки дисперсии (по второй формуле):",theor_2)

NameError: name 'lfd' is not defined

## Вычисление экспоненты

### Задания

1. Относительная ошибка приближения частичной суммой ряда Тейлора показательной функцией много больше для отрицательных аргументов. Объясните причину этого. Воспользуйтесь свойствами показательной функции, чтобы выравнить точность вычислений при положительных и отрицательных аргументах.
2. Почему абсолютная погрешность мала при аргументах близких к нулю? Как именно погрешность зависит от аргумента?
3. Абсолютная погрешность приближения функции частичной суммой ряда равна остатку этого ряда. Оцените остаток ряда Тейлора для экспоненты и найдите число слагаемых, необходимое для вычисления экспоненты с наперед заданной точностью. Проведите эксперимент и убедитесь, что предсказанная вами точность отличается от фактической не более чем на порядок.
4. Ошибка вычисления через частичную сумму складывается из ошибки отбрасывание остатка ряда и ошибки вычисления умножений и сложений. При увеличинии числа слагаемых первая ошибка уменьшается, но вторая растет. Для произвольного x оцените число слагаемых, при которых точность вычисления показательной функции максимальна.
5. Схема Горнера дает несколько меньшую погрешность, чем суммирование одночленов. Почему?
6. Можете предложить лучший способ вычисления показательной функции?

## Решение
## 1.


Ошибку можно объяснить, рассмотрев погрешность для рассматриваемого ряда:

$\bigg|\Delta\dfrac{x_n}{n!}\bigg| = \Delta\dfrac{|x^n|}{n!} = n\cdot\dfrac{|x^n|}{n!}\cdot |\delta x| = |\delta x|\dfrac{|x^n|}{(n-1)!}$


$\Delta S_N \leq \sum_{n=0}^N |\delta x|\dfrac{|x^n|}{(n-1)!} \leq |\delta x\cdot x| \sum_{n=0}^{N-1} \dfrac{|x^n|}{n!}\leq e^{|x|}\cdot |\Delta x|$


$|\delta S_N| = \dfrac{|\Delta x|e^{|x|}}{e^x} = |\Delta x| \cdot \exp[|x|-x]$


Из оценки погрешности видно, что при положительных значениях аргумента - экспонента не вносит вклад в погрешность (т.к. ее агрумент равен нулю), а при отрицательных значениях агрумент экспоненты равен $-2x$, что вносит сильный вклад в эволюцию ошибки - экспоненциально растущая ошибка при больишх значениях аргумента.